### Lab : Fine-Tuning Pre-trained Models for Climate Change Analysis

#### Problem Statement
The increasing frequency of wildfires has made it essential to develop accurate prediction models that can quickly identify fire-prone areas. This lab focuses on creating a wildfire detection model using satellite images. We fine-tune a pre-trained MobileNet model to classify satellite images into "Fire" and "No Fire" categories, which can contribute to early wildfire detection and prevention.



#### Objective
- To utilize transfer learning by fine-tuning the pre-trained MobileNet model for binary classification (Fire vs. No Fire).
- To preprocess and train the model using satellite images from the "Wildfire Prediction Dataset."
- To evaluate the model’s performance in identifying wildfire-prone areas based on satellite images.


#### System Requirements

**To Train the model in Local System**

**Hardware:**
- GPU (Recommended for faster training, e.g., Nvidia CUDA-compatible GPUs)
- RAM: 8 GB or more
- Storage: At least 5 GB of free space for datasets and model storage

**Software:**
- Python 3.x
- TensorFlow 2.x
- Pandas, NumPy
- Matplotlib (Optional for visualization)
- Jupyter Notebook or any Python IDE
- GPU drivers and CUDA (if available)

It is recommended to use google colab, so there will not be any need of setting up the environment.

#### Dataset
- Name: Wildfire Prediction Dataset
- Description: This dataset contains satellite images classified as either "Fire" or "No Fire." It is designed for training and testing models for wildfire prediction.
- Source: Kaggle: Wildfire Prediction Dataset


#### Code Explanation

**Dataset Download**<br>
The dataset is downloaded from Kaggle using the Kaggle API, which requires an API key.

In [ ]:
# prompt: download the kaggle dataset here:
# https://www.kaggle.com/datasets/abdelghaniaaba/wildfire-prediction-dataset
!pip install kaggle

# Upload your Kaggle API key (kaggle.json)
from google.colab import files
files.upload()

# Move the API key to the correct location
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download the dataset
!kaggle datasets download -d abdelghaniaaba/wildfire-prediction-dataset

# Unzip the dataset
!unzip wildfire-prediction-dataset.zip 


This section installs the Kaggle library and uses an uploaded Kaggle API key to access and download the dataset. It then unzips the dataset, making it available for use.

**Importing Libraries and Setting Parameters**<br>
The essential libraries and parameters for the model are loaded and initialized.

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

# Parameters
input_shape = (224, 224, 3)  # Standard input size for MobileNet
num_classes = 2  # Fire or No Fire
batch_size = 32
learning_rate = 0.001
epochs = 20


We set the input shape to 224x224 pixels, the required input for MobileNet. The batch size, learning rate, and the number of training epochs are also defined.



**Data Preprocessing and Augmentation**<br>
Image data is preprocessed, and augmentation techniques are applied to improve model robustness.

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255, 
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    'train',
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    'valid',
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)


The ImageDataGenerator class is used for rescaling and augmenting images. Augmentation techniques include rotation, width/height shifting, and flipping, helping to diversify the training dataset and reduce overfitting. The dataset is split into training and validation subsets.

**Load Pre-trained MobileNet Model**<br>
We load the MobileNet model with pre-trained weights from ImageNet and freeze the layers to retain existing knowledge.

In [ ]:
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=input_shape)

# Freeze the base model layers to prevent training
for layer in base_model.layers:
    layer.trainable = False


MobileNet is loaded without the top layer, as we’ll add a custom top layer for binary classification. Freezing layers ensures the pre-trained weights aren’t modified during training.

**Build the Model Architecture**<br>
The custom top layers are added to the MobileNet base to adapt it for the binary classification task.

In [ ]:
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),  # Optional fully connected layer
    Dense(1, activation='sigmoid')  # Binary classification (fire or no fire)
])


A global average pooling layer is added to reduce the spatial dimensions, followed by a dense layer with ReLU activation to introduce non-linearity. The final layer uses a sigmoid activation function for binary classification.



**Compile the Model**<br>
The model is compiled with the Adam optimizer and binary cross-entropy loss.

In [ ]:
model.compile(optimizer=Adam(learning_rate=learning_rate), 
              loss='binary_crossentropy', 
              metrics=['accuracy'])


The Adam optimizer is selected for adaptive learning rate capabilities, and binary cross-entropy is used as the loss function, appropriate for binary classification.

**Train the Model**<br>
The model is trained on the training data with validation after each epoch.

In [ ]:
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator
)


The fit function trains the model on the training data while validating on a separate set to track performance. Training for multiple epochs helps optimize the model weights.

**Model Evaluation and Saving**<br>
Evaluate the model’s performance and save the trained model for future use.

In [ ]:
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

model.save('forest_fire_detection_mobilenet.h5')


The model is evaluated on validation data, providing an accuracy metric to gauge its performance. The trained model is then saved in .h5 format, making it available for deployment or further fine-tuning.